Here, we test various parsers and approaches

## Dependencies

In [1]:
import os

from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain_community.embeddings import HuggingFaceEmbeddings 
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_community.llms import HuggingFaceEndpoint
from pathlib import Path
from PyPDF2 import PdfReader
from dotenv import load_dotenv, find_dotenv

from rich import print
from rich.pretty import pprint

from llama_parse import LlamaParse  


load_dotenv(find_dotenv('env'))

True

In [47]:
pdf1 = "data/test.pdf"  # a manually created simple pdf with a few words on 2 pages
pdf2 = "data/AMZN_Moodys_CreditRating_2023_p1.pdf" # first page, with the very difficult to read Exhibit 1 with a bar chart
pdf3 = "../../assignment_data/AMZN_Moodys_CreditRating_2023.pdf"

## PYPDFLOADER

In [4]:
# Load PDF document and create doc splits
def load_doc(list_file_path, chunk_size, chunk_overlap):
    # Processing for one document only
    # loader = PyPDFLoader(file_path)
    # pages = loader.load()
    loaders = [PyPDFLoader(x) for x in list_file_path]
    pages = []
    for loader in loaders:
        pages.extend(loader.load())
    # text_splitter = RecursiveCharacterTextSplitter(chunk_size = 600, chunk_overlap = 50)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size, 
        chunk_overlap = chunk_overlap)
    doc_splits = text_splitter.split_documents(pages)
    return doc_splits
splits = load_doc([pdf1], 600, 50)

print(splits)

[Document(page_content='���������������������������������', metadata={'source': 'data/test.pdf', 'page': 0})]


This is very bad since it's a very simple pdf with a few words, no images... but it works well on pdf2.

In [44]:
load_doc([pdf2], 600, 50)

[Document(page_content='CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdate\nRATINGS\nAmazon.com, Inc.\nDomicile Seattle, Washington,\nUnited States\nLong Term Rating A1\nType Senior Unsecured -\nDom Curr\nOutlook Stable\nPlease see the ratings section  at the end of this report\nfor more information. The ratings and outlook shown\nreflect information as of the publication date.\nContacts\nChristina Boni +1.212.553.0514\nSenior Vice President\nchristina.boni@moodys.com\nJack Myers +1.212.553.5116\nAssociate Analyst\njack.myers@moodys.com\nMargaret Taylor +1.212.553.0424\nAssociate Managing Director\nmargaret.taylor@moodys.comAmazon.com, Inc.', metadata={'source': 'data/AMZN_Moodys_CreditRating_2023_p1.pdf', 'page': 0}),
 Document(page_content="margaret.taylor@moodys.comAmazon.com, Inc.\nUpdate to credit analysis\nSummary\nAmazon.com, Inc. 's (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\nis synonymous with online retail, as well as the strength and profitabil

In [46]:
loaders = [PyPDFLoader(x) for x in [pdf2]]
pages = []
for loader in loaders:
    pages.extend(loader.load())
pprint(pages)

[
│   Document(
│   │   page_content="CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdate\nRATINGS\nAmazon.com, Inc.\nDomicile Seattle, Washington,\nUnited States\nLong Term Rating A1\nType Senior Unsecured -\nDom Curr\nOutlook Stable\nPlease see the ratings section  at the end of this report\nfor more information. The ratings and outlook shown\nreflect information as of the publication date.\nContacts\nChristina Boni +1.212.553.0514\nSenior Vice President\nchristina.boni@moodys.com\nJack Myers +1.212.553.5116\nAssociate Analyst\njack.myers@moodys.com\nMargaret Taylor +1.212.553.0424\nAssociate Managing Director\nmargaret.taylor@moodys.comAmazon.com, Inc.\nUpdate to credit analysis\nSummary\nAmazon.com, Inc. 's (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\nis synonymous with online retail, as well as the strength and profitability of Amazon Web\nServices (“AWS”), the market leader in the cloud computing market. The company is reliant\non the operating income derived from AWS, as its non-AWS profitability has remained weak\nsince the end of 2021. Although the company is making progress with improving productivity\nand reducing costs, online operating margins remain well below historical levels. Amazon has\ntaken actions to make its fulfillment operations more efficient as its business grows into its\ncapacity, which doubled during the pandemic. Amazon has also built a solid ecosystem of\nentertainment content that enhances its offering, operates a formidable third-party seller\nbusiness and generates a solid and growing revenue stream from advertising. Nonetheless,\nits credit metrics are currently weak for the A1 rating with RCF/Debt below 50%, as lower\nprofitability, coupled with increased levels of investment have led to higher debt levels\nand lower cash balances. Capital allocation will be critical to improving its credit profile as\nAmazon navigates a weaker economic backdrop that could dampen demand for its products\nand services as it pursues cost reductions and efficiencies to restore profitability at online\nretail. The growing online presence of brick-and-mortar retailers, as well as the increasing\ncompetition from larger, well capitalized companies in AWS' universe also presents future\nchallenges.\nExhibit 1\nAmazon's debt has continued to rise as operating income remains below 2019\n$0$20,000$40,000$60,000$80,000$100,000$120,000$140,000$160,000$180,000\n$0$5,000$10,000$15,000$20,000$25,000$30,000\n2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 Q1 -23 LTM\nMoody's Adjusted Debt (USD Millions)Moody's Adj. Operating Income (USD Millions)Moody's adjusted operating income Moody's adjusted debt\nDebt includes lease\nSource: Moody’s Financial Metrics™\n",
│   │   metadata={'source': 'data/AMZN_Moodys_CreditRating_2023_p1.pdf', 'page': 0}
│   )
]

In [48]:
loaders = [PyPDFLoader(x) for x in [pdf3]]
pages = []
for loader in loaders:
    pages.extend(loader.load())
pprint(pages)

[
│   Document(
│   │   page_content="CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdate\nRATINGS\nAmazon.com, Inc.\nDomicile Seattle, Washington,\nUnited States\nLong Term Rating A1\nType Senior Unsecured -\nDom Curr\nOutlook Stable\nPlease see the ratings section  at the end of this report\nfor more information. The ratings and outlook shown\nreflect information as of the publication date.\nContacts\nChristina Boni +1.212.553.0514\nSenior Vice President\nchristina.boni@moodys.com\nJack Myers +1.212.553.5116\nAssociate Analyst\njack.myers@moodys.com\nMargaret Taylor +1.212.553.0424\nAssociate Managing Director\nmargaret.taylor@moodys.comAmazon.com, Inc.\nUpdate to credit analysis\nSummary\nAmazon.com, Inc. 's (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\nis synonymous with online retail, as well as the strength and profitability of Amazon Web\nServices (“AWS”), the market leader in the cloud computing market. The company is reliant\non the operating income derived from AWS, as its non-AWS profitability has remained weak\nsince the end of 2021. Although the company is making progress with improving productivity\nand reducing costs, online operating margins remain well below historical levels. Amazon has\ntaken actions to make its fulfillment operations more efficient as its business grows into its\ncapacity, which doubled during the pandemic. Amazon has also built a solid ecosystem of\nentertainment content that enhances its offering, operates a formidable third-party seller\nbusiness and generates a solid and growing revenue stream from advertising. Nonetheless,\nits credit metrics are currently weak for the A1 rating with RCF/Debt below 50%, as lower\nprofitability, coupled with increased levels of investment have led to higher debt levels\nand lower cash balances. Capital allocation will be critical to improving its credit profile as\nAmazon navigates a weaker economic backdrop that could dampen demand for its products\nand services as it pursues cost reductions and efficiencies to restore profitability at online\nretail. The growing online presence of brick-and-mortar retailers, as well as the increasing\ncompetition from larger, well capitalized companies in AWS' universe also presents future\nchallenges.\nExhibit 1\nAmazon's debt has continued to rise as operating income remains below 2019\n$0$20,000$40,000$60,000$80,000$100,000$120,000$140,000$160,000$180,000\n$0$5,000$10,000$15,000$20,000$25,000$30,000\n2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 Q1 -23 LTM\nMoody's Adjusted Debt (USD Millions)Moody's Adj. Operating Income (USD Millions)Moody's adjusted operating income Moody's adjusted debt\nDebt includes lease\nSource: Moody’s Financial Metrics™\n",
│   │   metadata={'source': '../../assignment_data/AMZN_Moodys_CreditRating_2023.pdf', 'page': 0}
│   ),
│   Document(
│   │   page_content="MOODY'S INVESTORS SERVICE CORPORATES\nCredit strengths\n»Leading online retailer and cloud provider\n»AWS provides an increasing income stream provides solid positioning in AI\n»Prime membership base supports customer loyalty\n»Advertising revenue remains a significant area of growth\nCredit challenges\n»Brick-and-mortar retailers continue to increase online retail presence\n»Heightened cloud competition from larger, well capitalized tech companies\n»Inefficiencies in its retail fulfillment operations weigh on profitability\n»Reduced cash balance lowers cushion for volatile or heavy investment periods\nRating outlook\nThe stable outlook reflects our view that Amazon will quickly restore its credit metrics to levels reflective of its A1 rating. The outlook\nalso assumes that Amazon will maintain excellent liquidity and consistent financial strategies.\nFactors that could lead to an upgrade\n»Ratings could be upgraded if Amazon's numerous investments generate commensurate levels of profitability such that RCF/debt is\nmaintained around 65%.\n»Additional factors that would be critical for an upgrade ar

In [49]:
pages[0].page_content

"CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdate\nRATINGS\nAmazon.com, Inc.\nDomicile Seattle, Washington,\nUnited States\nLong Term Rating A1\nType Senior Unsecured -\nDom Curr\nOutlook Stable\nPlease see the ratings section  at the end of this report\nfor more information. The ratings and outlook shown\nreflect information as of the publication date.\nContacts\nChristina Boni +1.212.553.0514\nSenior Vice President\nchristina.boni@moodys.com\nJack Myers +1.212.553.5116\nAssociate Analyst\njack.myers@moodys.com\nMargaret Taylor +1.212.553.0424\nAssociate Managing Director\nmargaret.taylor@moodys.comAmazon.com, Inc.\nUpdate to credit analysis\nSummary\nAmazon.com, Inc. 's (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\nis synonymous with online retail, as well as the strength and profitability of Amazon Web\nServices (“AWS”), the market leader in the cloud computing market. The company is reliant\non the operating income derived from AWS, as its non-AWS pro

## PyPDF2

In [15]:
reader = PdfReader(pdf1)
page = reader.pages[0]
print(page.extract_text())

���������������������������������

In [16]:
# https://pypdf2.readthedocs.io/en/3.x/user/extract-text.html
reader = PdfReader(pdf1)
number_of_pages = len(reader.pages)
print(f"Number of pages: {number_of_pages}")
page = reader.pages[0]
text = page.extract_text()
print(text)

Number of pages: 2

���������������������������������

In [17]:
# extract the images
reader = PdfReader(pdf2)
page = reader.pages[0]
count = 0

for image_file_object in page.images:
    with open(str(count) + image_file_object.name, "wb") as fp:
        fp.write(image_file_object.data)
        count += 1

OSError: cannot write mode PA as PNG

Total disaster with the library's own code.  PyPDF2 is not just a parser, it can merge etc so maybe we need something more specialized at parsing.

## LLAMA_PARSE

In [5]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()
#%%
# https://github.com/run-llama/llama_parse
parser = LlamaParse(
    api_key = os.getenv("LLAMA_PARSE_API_KEY"),
    num_workers=4,
    verbose=True,
    language="en",
    result_type="text"  # "markdown" and "text" are available
)
#%%
documents = parser.load_data(pdf1)
# documents = parser.load_data([pdf1, pdf2])   # sync batch

# documents = await parser.aload_data(pdf1)  # async
# documents = await parser.aload_data([pdf1, pdf2]) # async batch
print(documents)

Started parsing the file under job_id a3a95967-3301-4140-9c89-e1f5d4f0677d
[Document(id_='3f10f65a-c636-4df9-83b1-1ffb0878bb70', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='A very simple pdf file\n\n\nsecond line\n---\n\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')]


In [8]:
# https://github.com/run-llama/llama_parse/blob/main/examples/demo_json.ipynb
docs = parser.get_json_result(pdf1)
docs

Started parsing the file under job_id f34a3133-b632-4ddd-8035-eacbe2678796


[{'pages': [{'page': 1,
    'text': 'A very simple pdf file\n\n\nsecond line',
    'md': 'A very simple pdf file\n\nsecond line',
    'images': [],
    'items': [{'type': 'text',
      'value': 'A very simple pdf file\n\nsecond line',
      'md': 'A very simple pdf file\n\nsecond line'}]},
   {'page': 2,
    'text': 'text on 2nd page\n\n\nurl',
    'md': 'text on 2nd page\n\n\nurl',
    'images': [],
    'items': [{'type': 'text',
      'value': 'text on 2nd page\n\n\nurl',
      'md': 'text on 2nd page\n\n\nurl'}]}],
  'job_id': 'f34a3133-b632-4ddd-8035-eacbe2678796',
  'file_path': 'data/test.pdf'}]

In [9]:
docs[0]['pages'][0]['text']

'A very simple pdf file\n\n\nsecond line'

In [10]:
docs[0]['pages'][1]['text']

'text on 2nd page\n\n\nurl'

In [11]:
docs2 = parser.get_json_result(pdf2)
docs2[0]['pages'][0]['text']

Started parsing the file under job_id 4e1922f5-566b-4bf7-8684-da7f4a977e37


"                                                                                                                                                                                          CORPORATES\n\n\n                CREDIT OPINION                                            Amazon.com, Inc.\n                23 May 2023\nMoody's Adj. Operating Income (USD Millions)                              Update to credit analysis\n                  Update                                                  Summary\n                                                                          Amazon.com, Inc.'s (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\n                           Send Your Feedback                             is synonymous with online retail, as well as the strength and profitability of Amazon Web\n                                                                          Services (“AWS”), the market leader in the cloud computing market. The company is 

In [13]:
pprint(docs2[0])

{
│   'pages': [
│   │   {
│   │   │   'page': 1,
│   │   │   'text': "                                                                                                                                                                                          CORPORATES\n\n\n                CREDIT OPINION                                            Amazon.com, Inc.\n                23 May 2023\nMoody's Adj. Operating Income (USD Millions)                              Update to credit analysis\n                  Update                                                  Summary\n                                                                          Amazon.com, Inc.'s (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\n                           Send Your Feedback                             is synonymous with online retail, as well as the strength and profitability of Amazon Web\n                                                                          Services (“AWS”), the market leader in the cloud computing market. The company is reliant\n                                                                          on the operating income derived from AWS, as its non-AWS profitability has remained weak\n                                                                          since the end of 2021. Although the company is making progress with improving productivity\n                RATINGS                                                   and reducing costs, online operating margins remain well below historical levels. Amazon has\n                 Amazon.com, Inc.                                         taken actions to make its fulfillment operations more efficient as its business grows into its\n                 Domicile                  Seattle, Washington,\n                                           United States                  capacity, which doubled during the pandemic. Amazon has also built a solid ecosystem of\n                 Long Term Rating          A1                             entertainment content that enhances its offering, operates a formidable third-party seller\n                 Type                      Senior Unsecured -             business and generates a solid and growing revenue stream from advertising. Nonetheless,\n                                           Dom Curr\n                 Outlook                   Stable                         its credit metrics are currently weak for the A1 rating with RCF/Debt below 50%, as lower\n                                                                          profitability, coupled with increased levels of investment have led to higher debt levels\n                Please see the ratings section at the end of this report  and lower cash balances. Capital allocation will be critical to improving its credit profile as\n                for more information. The ratings and outlook shown       Amazon navigates a weaker economic backdrop that could dampen demand for its products\n                reflect information as of the publication date.           and services as it pursues cost reductions and efficiencies to restore profitability at online\n                                                                          retail. The growing online presence of brick-and-mortar retailers, as well as the increasing\n                Contacts                                                  competition from larger, well capitalized companies in AWS' universe also presents future\n                Christina Boni                   +1.212.553.0514          challenges.\n                Senior Vice President                                     Exhibit 1\n                christina.boni@moodys.com                                 Amazon's debt has continued to rise as operating income remains below 2019\n\n\n                Jack Myers                        +1.212.553.5116               $30,000       Moody's adjusted operating incomeMoody's adjusted debt   

Quite good but it can't read the bar graph in Exhibit1

In [ ]:
# one can use SimpleDirectoryReader to load files from a directory
from llama_index.core import SimpleDirectoryReader  # pip install llama-index

file_extractor = {".pdf": parser}
reader = SimpleDirectoryReader("./data", file_extractor=file_extractor)
documents = reader.load_data()

## PYMUPDF

In [23]:
# https://pymupdf.readthedocs.io/en/latest/rag.html
from llama_index.readers.file import PyMuPDFReader
loader = PyMuPDFReader()
documents = loader.load(file_path=pdf1)
pprint(documents)

[
│   Document(
│   │   id_='a66aa726-5a25-4a0b-90f6-74b4f3af1599',
│   │   embedding=None,
│   │   metadata={'total_pages': 2, 'file_path': 'data/test.pdf', 'source': '1'},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='A very simple pdf file\nsecond line\n',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   ),
│   Document(
│   │   id_='44d08ca7-e6f2-4f9d-828e-4dfc4c5d1281',
│   │   embedding=None,
│   │   metadata={'total_pages': 2, 'file_path': 'data/test.pdf', 'source': '2'},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text='text on 2nd page\nurl\nText in annotation box\n',
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   )
]

In [24]:
documents = loader.load(file_path=pdf2)
pprint(documents)

[
│   Document(
│   │   id_='fe0315af-5cec-4c78-a0b7-479c5a44de7f',
│   │   embedding=None,
│   │   metadata={'total_pages': 1, 'file_path': 'data/AMZN_Moodys_CreditRating_2023_p1.pdf', 'source': '1'},
│   │   excluded_embed_metadata_keys=[],
│   │   excluded_llm_metadata_keys=[],
│   │   relationships={},
│   │   text="CORPORATES\nCREDIT OPINION\n23 May 2023\nUpdate\nRATINGS\nAmazon.com, Inc.\nDomicile\nSeattle, Washington,\nUnited States\nLong Term Rating\nA1\nType\nSenior Unsecured -\nDom Curr\nOutlook\nStable\nPlease see the ratings section at the end of this report\nfor more information. The ratings and outlook shown\nreflect information as of the publication date.\nContacts\nChristina Boni\n+1.212.553.0514\nSenior Vice President\nchristina.boni@moodys.com\nJack Myers\n+1.212.553.5116\nAssociate Analyst\njack.myers@moodys.com\nMargaret Taylor\n+1.212.553.0424\nAssociate Managing Director\nmargaret.taylor@moodys.com\nAmazon.com, Inc.\nUpdate to credit analysis\nSummary\nAmazon.com, Inc.'s (A1/Prime-1 stable) credit profile reflects its powerful global brand, which\nis synonymous with online retail, as well as the strength and profitability of Amazon Web\nServices (“AWS”), the market leader in the cloud computing market. The company is reliant\non the operating income derived from AWS, as its non-AWS profitability has remained weak\nsince the end of 2021. Although the company is making progress with improving productivity\nand reducing costs, online operating margins remain well below historical levels. Amazon has\ntaken actions to make its fulfillment operations more efficient as its business grows into its\ncapacity, which doubled during the pandemic. Amazon has also built a solid ecosystem of\nentertainment content that enhances its offering, operates a formidable third-party seller\nbusiness and generates a solid and growing revenue stream from advertising. Nonetheless,\nits credit metrics are currently weak for the A1 rating with RCF/Debt below 50%, as lower\nprofitability, coupled with increased levels of investment have led to higher debt levels\nand lower cash balances. Capital allocation will be critical to improving its credit profile as\nAmazon navigates a weaker economic backdrop that could dampen demand for its products\nand services as it pursues cost reductions and efficiencies to restore profitability at online\nretail. The growing online presence of brick-and-mortar retailers, as well as the increasing\ncompetition from larger, well capitalized companies in AWS' universe also presents future\nchallenges.\nExhibit 1\nAmazon's debt has continued to rise as operating income remains below 2019\n$0\n$20,000\n$40,000\n$60,000\n$80,000\n$100,000\n$120,000\n$140,000\n$160,000\n$180,000\n$0\n$5,000\n$10,000\n$15,000\n$20,000\n$25,000\n$30,000\n2013\n2014\n2015\n2016\n2017\n2018\n2019\n2020\n2021\n2022\nQ1 -23 LTM\nMoody's Adjusted Debt (USD Millions)\nMoody's Adj. Operating Income (USD Millions)\nMoody's adjusted operating income\nMoody's adjusted debt\nDebt includes lease\nSource: Moody’s Financial Metrics™\n",
│   │   start_char_idx=None,
│   │   end_char_idx=None,
│   │   text_template='{metadata_str}\n\n{content}',
│   │   metadata_template='{key}: {value}',
│   │   metadata_seperator='\n'
│   )
]

Not bad but it reads the axis values of Exhibit1, and not the bar graph itself

## PDFPLUMBER

In [50]:
from langchain.document_loaders.parsers.pdf import PDFPlumberParser


# look into the library, plenty of options and other libraries 

# Initialize the parser
parser = PDFPlumberParser()

# Load your PDF data
data = parser.load(pdf1)

# Now you can process the data
processed_data = parser.process(data)
pprint(processed_data)

AttributeError: 'PDFPlumberParser' object has no attribute 'load'

In [36]:
import pdfplumber

In [43]:
with pdfplumber.open(pdf2) as pdf:
    first_page = pdf.pages[0]
    pprint(first_page.lines)

[
│   {
│   │   'x0': 27.0,
│   │   'y0': 645.856,
│   │   'x1': 585.0,
│   │   'y1': 645.856,
│   │   'width': 558.0,
│   │   'height': 0.0,
│   │   'pts': [(27.0, 146.144), (585.0, 146.144)],
│   │   'linewidth': 4.002,
│   │   'stroke': True,
│   │   'fill': False,
│   │   'evenodd': False,
│   │   'stroking_color': (1, 0.6, 0, 0),
│   │   'non_stroking_color': None,
│   │   'mcid': None,
│   │   'tag': None,
│   │   'object_type': 'line',
│   │   'page_number': 1,
│   │   'stroking_pattern': None,
│   │   'non_stroking_pattern': None,
│   │   'path': [('m', (27.0, 146.144)), ('l', (585.0, 146.144))],
│   │   'dash': ([], 0),
│   │   'top': 146.144,
│   │   'bottom': 146.144,
│   │   'doctop': 146.144
│   },
│   {
│   │   'x0': 27.0,
│   │   'y0': 645.856,
│   │   'x1': 585.0,
│   │   'y1': 645.856,
│   │   'width': 558.0,
│   │   'height': 0.0,
│   │   'pts': [(27.0, 146.144), (585.0, 146.144)],
│   │   'linewidth': 4.002,
│   │   'stroke': True,
│   │   'fill': False,
│   │   'evenodd': False,
│   │   'stroking_color': (1, 0.6, 0, 0),
│   │   'non_stroking_color': None,
│   │   'mcid': None,
│   │   'tag': None,
│   │   'object_type': 'line',
│   │   'page_number': 1,
│   │   'stroking_pattern': None,
│   │   'non_stroking_pattern': None,
│   │   'path': [('m', (27.0, 146.144)), ('l', (585.0, 146.144))],
│   │   'dash': ([], 0),
│   │   'top': 146.144,
│   │   'bottom': 146.144,
│   │   'doctop': 146.144
│   },
│   {
│   │   'x0': 45.0,
│   │   'y0': 343.268025,
│   │   'x1': 189.0,
│   │   'y1': 343.268025,
│   │   'width': 144.0,
│   │   'height': 0.0,
│   │   'pts': [(45.0, 448.731975), (189.0, 448.731975)],
│   │   'linewidth': 0.149994,
│   │   'stroke': True,
│   │   'fill': False,
│   │   'evenodd': False,
│   │   'stroking_color': (0, 0, 0, 0.62),
│   │   'non_stroking_color': (0.6, 0.4, 0.4, 1),
│   │   'mcid': None,
│   │   'tag': None,
│   │   'object_type': 'line',
│   │   'page_number': 1,
│   │   'stroking_pattern': None,
│   │   'non_stroking_pattern': None,
│   │   'path': [('m', (45.0, 448.731975)), ('l', (189.0, 448.731975))],
│   │   'dash': ([], 0),
│   │   'top': 448.731975,
│   │   'bottom': 448.731975,
│   │   'doctop': 448.731975
│   },
│   {
│   │   'x0': 526.4859179252,
│   │   'y0': 188.08087635180004,
│   │   'x1': 526.4859179252,
│   │   'y1': 270.55985592300004,
│   │   'width': 0.0,
│   │   'height': 82.4789795712,
│   │   'pts': [(526.4859179252, 603.9191236482), (526.4859179252, 521.440144077)],
│   │   'linewidth': 0.2279,
│   │   'stroke': True,
│   │   'fill': False,
│   │   'evenodd': False,
│   │   'stroking_color': (0.843, 0.855, 0.839),
│   │   'non_stroking_color': (0, 0.592, 0.459),
│   │   'mcid': None,
│   │   'tag': 'Artifact',
│   │   'object_type': 'line',
│   │   'page_number': 1,
│   │   'stroking_pattern': None,
│   │   'non_stroking_pattern': None,
│   │   'path': [('m', (526.4859179252, 603.9191236482)), ('l', (526.4859179252, 521.440144077))],
│   │   'dash': None,
│   │   'top': 521.440144077,
│   │   'bottom': 603.9191236482,
│   │   'doctop': 521.440144077
│   },
│   {
│   │   'x0': 241.102111912,
│   │   'y0': 188.08087635180004,
│   │   'x1': 241.102111912,
│   │   'y1': 270.55985592300004,
│   │   'width': 0.0,
│   │   'height': 82.4789795712,
│   │   'pts': [(241.102111912, 603.9191236482), (241.102111912, 521.440144077)],
│   │   'linewidth': 0.2279,
│   │   'stroke': True,
│   │   'fill': False,
│   │   'evenodd': False,
│   │   'stroking_color': (0.843, 0.855, 0.839),
│   │   'non_stroking_color': (0, 0.592, 0.459),
│   │   'mcid': None,
│   │   'tag': 'Artifact',
│   │   'object_type': 'line',
│   │   'page_number': 1,
│   │   'stroking_pattern': None,
│   │   'non_stroking_pattern': None,
│   │   'path': [('m', (241.102111912, 603.9191236482)), ('l', (241.102111912, 521.440144077))],
│   │   'dash': None,
│   │   'top': 521.440144077,
│   │   'bottom': 603.9191236482,
│   │   'doctop': 521.440144077
│   },
│   {
│   │   'x0': 241.102111912,
│   │   'y0': 188

## LLAVA
Partition PDF tables, text, and images
LLaVA Paper: https://arxiv.org/pdf/2304.08485.pdf

In [35]:
# pip install "pip install unstructured[all-docs]"
from unstructured.partition.pdf import partition_pdf

# Path to save images
path = "data"

# Get elements
raw_pdf_elements = partition_pdf(
    filename=pdf1,
    # Using pdf format to find embedded image blocks
    extract_images_in_pdf=True,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    infer_table_structure=True,
    # Post processing to aggregate text once we have the title
    chunking_strategy="by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    # Hard max on chunks
    max_characters=4000,
    new_after_n_chars=3800,
    combine_text_under_n_chars=2000,
    image_output_dir_path=path,
)
pprint(raw_pdf_elements)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


NameError: name 'sort_page_elements' is not defined

Impossible to make it work, it's error 'something is missing', I brew/pip install it, then a new error, 10 times over.  I pip installed unstructured[all-docs] as requested in the article, and yet there's sthg missing ... I give up on this.

## TABULA

For tables... https://github.com/tabulapdf/tabula

## CHARTS

https://www.researchgate.net/publication/372616217_Automatic_Chart_Understanding_a_Review

https://ieeexplore.ieee.org/document/9599112